#Import Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

from pandas import read_csv
from pandas import DataFrame
from pandas import concat

from pywt import dwt
from pywt import idwt
import pywt

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

from keras.layers import LSTM, Dense
from keras.models import Sequential
from tensorflow.keras import initializers

from openpyxl.workbook import Workbook
import pickle

# Fungsi ini menetapkan nilai seed menjadi 42 untuk semua operasi acak yang dilakukan oleh TensorFlow.
tf.random.set_seed(42)

#Class Preprocessing

In [ ]:
class Preprocessing:
        #Melakukan fungsi MinMaxScaler sesuai dengan rumus 2.11
    def minmax_scale(df):
        # Pemilihan fitur yang akan digunakan dan memindahkan kolom close ke paling kanan
        FEATURES = ['Open', 'High', 'Low', 'Volume', 'Close']
        # Memanggil objek MinMaxScaler
        scaler = MinMaxScaler()
        # Memasukan hasil dari minmax ke df_scaled
        df_scaled = scaler.fit_transform(np.array(df[FEATURES]))

        scaler_pred = MinMaxScaler()
        return df_scaled, scaler_pred

        # Melakukan proses denormalisasi
    def inverse_scaler(pred, scaler):
        pred_inversed = scaler.inverse_transform(pred)
        return pred_inversed

        # Melakukan proses splitting data
    def splitting_data(df):
        # Mengatur ukuran test data sebanyak 3
        test_size = 3
        # Mengatur ukuran train data sebanyak jumlah data - 3
        train_size = len(df)-test_size
        # Membagi data sesuai dengan train dan test size
        train_data,test_data= df[0:train_size,:], df[train_size:len(df),:]
        return train_data, test_data


    def data_denoising(data,wavelet_type, threshold):
        # Membuat DataFrame kosong
        decomposed_data = pd.DataFrame(index=range(len(data)))
        for i in range(data.shape[1]):
            # Mendekomposisi data menggunakan pywt.dwt sesuai dengan rumus 2.13 dan 2.14
            cA, cD = pywt.dwt(data[:, i], wavelet_type)

            # Melakukan thresholding pada cD sesuai dengan rumus 2.15
            cD_threshold = pywt.threshold(cD, threshold, mode='soft')

            # Menggabungkan cA dan cD_threshold untuk merekonstruksi
            reconstructed_data = pywt.idwt(cA, cD_threshold, wavelet_type)

            # Menyimpan hasil rekonstruksi ke dalam dataframe hasil
            decomposed_data[i] = reconstructed_data

        return decomposed_data


    def create_dataset(dataset, time_step=1, index=4):
      # Membuat 2 array kosong sebagai data input dan target
        dataX = []
        dataY = []

        for i in range(len(dataset)-time_step):
            # Menetapkan nilai input
            dataX.append(dataset[i:(i+time_step)])
            # Menetapkan nilai close pada time-step selanjutnya sebagai target
            dataY.append(float(dataset[i+time_step][index]))
        return np.array(dataX), np.array(dataY)


#Class Model

In [ ]:
class NeuralNetwork:
    def train_lstm(train_X, train_y, test_X, test_y, unit, epoch, batch):
        # Memanggil objek Sequential dari library keras
        model = Sequential()

        # Menambahkan layer LSTM dengan unit yang ditentukan
        # kernel_initializer diatur dengan GlorotUniform untuk inisialisasi bobot,
        # seed = 42 artinya memastikan bahwa bobot layer diinisialisasi dengan nilai yang sama setiap kali kode dijalankan.
        # Nilai 42 adalah nilai yang paling sering digunakan karena sifatnya yang sederhana mengacu pada penelitian Douglas Adams' “The Hitchhiker's Guide to the Galaxy,”
        # input_shape disesuaikan dengan bentuk dari train_X
        model.add(LSTM(unit, kernel_initializer=initializers.GlorotUniform(seed=42),
                       input_shape=(train_X.shape[1], train_X.shape[2])))

        # Menambahkan layer Dense (fully connected layer) dengan satu unit output
        # kernel_initializer diatur dengan GlorotUniform untuk inisialisasi bobot
        model.add(Dense(units=1, kernel_initializer=initializers.GlorotUniform(seed=42)))

        # Melakukan compile pada model dengan loss function 'mae' (mean absolute error) sesuai pada rumus 2.18 dan optimizer 'adam' sesuai dengan algoritma 2.1
        # Learning rate yang digunakan adalah default yang bernilai 0.001 sesuai dengan yang tertera pada dokumentasi library keras
        model.compile(loss='mae',optimizer='adam')

        # Melatih model dengan data train_X dan train_y
        # Menggunakan jumlah epoch, batch_size yang ditentukan
        # Menyediakan data validasi (test_X, test_y)
        history = model.fit(train_X, train_y, epochs=epoch, batch_size=batch,
                            validation_data=(test_X, test_y), verbose=2, shuffle=False)

        return model, history

    def save_model(model, category, stocks_name, hyperparam):
        if category == 0:
            model.save('LSTM_'+ stocks_name +str(hyperparam)+'.h5')
        else:
            model.save('DWT_LSTM'+ stocks_name +str(hyperparam)+'.h5')

#Class Evaluation

In [ ]:
class Evaluation:
    # Fungsi untuk menghitung RMSE sesuai dengan rumus 2.17
    def rmse(y, yhat):
        # Menghitung perbedaan antara nilai aktual dan nilai prediksi
        differences = [y[i] - yhat[i] for i in range(len(y))]
        # Mengkuadratkan setiap perbedaan
        squared_differences = [d**2 for d in differences]
        # Menjumlahkan semua perbedaan kuadrat
        sum_squared_differences = sum(squared_differences)
        # Menghitung rata-rata dari perbedaan kuadrat
        mean_squared_error = sum_squared_differences / len(y)
        # Mengembalikan akar kuadrat dari rata-rata perbedaan kuadrat
        return (mean_squared_error**0.5)[0]

    # Fungsi untuk menghitung MAE sesuai dengan rumus 2.18
    def mae(y, yhat):
        # Menghitung perbedaan antara nilai aktual dan nilai prediksi
        differences = [y[i] - yhat[i] for i in range(len(y))]
        # Mengambil nilai absolut dari setiap perbedaan
        absolute_differences = [abs(x) for x in differences]
        # Menjumlahkan semua perbedaan absolut
        sum_absolute_difference = sum(absolute_differences)
        # Menghitung rata-rata dari perbedaan absolut
        mean_absolute_error = sum_absolute_difference / len(y)
        # Mengembalikan rata-rata dari perbedaan absolut
        return mean_absolute_error[0]

    # Fungsi untuk menghitung MAPE sesuai dengan rumus 2.19
    def mape(y, yhat):
        # Menghitung perbedaan absolut yang dibagi dengan nilai aktual
        divided_differences = [abs((y[i] - yhat[i])/y[i]) for i in range(len(y))]
        # Menjumlahkan semua perbedaan absolut yang dibagi
        sum_absolute_difference = sum(divided_differences)
        # Menghitung rata-rata dari perbedaan absolut yang dibagi
        mean_absolute_percentage_error = sum_absolute_difference / len(y)
        # Mengembalikan rata-rata persentase kesalahan absolut dikalikan 100
        return (mean_absolute_percentage_error*100)[0]

#Main

In [ ]:
# Membaca dataframe melalui google drive
BBCA_dfd= pd.read_csv('/content/drive/MyDrive/Data/BBCA.JK.csv')
BBCA_dfd

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-16,5260.0,5285.0,5245.0,5285.0,4813.968262,94972500
1,2019-01-17,5290.0,5330.0,5285.0,5330.0,4854.957520,91654000
2,2019-01-18,5340.0,5425.0,5315.0,5425.0,4941.490234,100873500
3,2019-01-21,5425.0,5600.0,5390.0,5545.0,5050.795410,87118000
4,2019-01-22,5600.0,5620.0,5450.0,5600.0,5100.893066,91907000
...,...,...,...,...,...,...,...
1226,2024-01-09,9600.0,9625.0,9575.0,9625.0,9625.000000,59848600
1227,2024-01-10,9650.0,9650.0,9550.0,9550.0,9550.000000,52774900
1228,2024-01-11,9625.0,9650.0,9575.0,9575.0,9575.000000,39381500
1229,2024-01-12,9650.0,9700.0,9600.0,9700.0,9700.000000,68253400


##Data Normalization

In [ ]:
# Melakukan Nomalisasi data
x,scaler = Preprocessing.minmax_scale(BBCA_dfd)

In [ ]:
print('Data kelima fitur yang dinormalisasi : ')
print(x)

Data kelima fitur yang dinormalisasi : 
[[0.15428571 0.09897959 0.17116279 0.1313902  0.16223909]
 [0.16       0.10816327 0.17860465 0.1267992  0.17077799]
 [0.16952381 0.12755102 0.18418605 0.13955397 0.18880455]
 ...
 [0.98571429 0.98979592 0.97674419 0.05448254 0.97628083]
 [0.99047619 1.         0.98139535 0.09442552 1.        ]
 [1.         1.         1.         0.         1.        ]]


## Data Splitting

In [ ]:
# Melakukan splitting data
train,test = Preprocessing.splitting_data(x)

In [ ]:
print("Training Data :")
print(train)
print("=========================================================")
print("Testing Data :")
print(test)

Training Data :
[[0.15428571 0.09897959 0.17116279 0.1313902  0.16223909]
 [0.16       0.10816327 0.17860465 0.1267992  0.17077799]
 [0.16952381 0.12755102 0.18418605 0.13955397 0.18880455]
 ...
 [0.98095238 0.98469388 0.95813953 0.07041233 0.97628083]
 [0.98095238 0.98469388 0.97674419 0.08279786 0.9857685 ]
 [0.99047619 0.98979592 0.97209302 0.07301171 0.971537  ]]
Testing Data :
[[0.98571429 0.98979592 0.97674419 0.05448254 0.97628083]
 [0.99047619 1.         0.98139535 0.09442552 1.        ]
 [1.         1.         1.         0.         1.        ]]


##Data Denoising

In [ ]:
# Menginisialisasi threshold dan jenis wavelet yang akan digunakan
threshold = 0.004
wavelet_type = 'haar'
#  Melakukan proses denoising data menggunakan DWT
denoised_train = Preprocessing.data_denoising(train,wavelet_type,threshold)

In [ ]:
print("Hasil data training yang telah bersih dari noise ")
print(denoised_train.values)
print(len(denoised_train.values))

Hasil data training yang telah bersih dari noise 
[[0.15711414 0.10180802 0.17399122 0.1290947  0.16506752]
 [0.15717157 0.10533484 0.17577622 0.1290947  0.16794956]
 [0.17235224 0.13037945 0.18701447 0.13672554 0.19163298]
 ...
 [0.97812395 0.98214286 0.95813953 0.07324076 0.97628083]
 [0.98378081 0.9872449  0.9744186  0.07996943 0.98294007]
 [0.98764776 0.9872449  0.9744186  0.07584014 0.97436543]]
1228


##Creating Dataset

In [ ]:
# Membagi data train dan test menjadi input dan target
train_X, train_y = Preprocessing.create_dataset(denoised_train.values)
test_X, test_y = Preprocessing.create_dataset(test)

In [ ]:
print("Data train_X")
print("=====================================")
print(train_X)

Data train_X
[[[0.15711414 0.10180802 0.17399122 0.1290947  0.16506752]]

 [[0.15717157 0.10533484 0.17577622 0.1290947  0.16794956]]

 [[0.17235224 0.13037945 0.18701447 0.13672554 0.19163298]]

 ...

 [[0.96473319 0.98214286 0.95813953 0.1151666  0.97628083]]

 [[0.97812395 0.98214286 0.95813953 0.07324076 0.97628083]]

 [[0.98378081 0.9872449  0.9744186  0.07996943 0.98294007]]]


In [ ]:
print("Data train_y")
print("=====================================")
print(train_y)

Data train_y
[0.16794956 0.19163298 0.20874653 ... 0.97628083 0.98294007 0.97436543]


In [ ]:
print("Data test_X")
print("=====================================")
print(test_X)

Data test_X
[[[0.98571429 0.98979592 0.97674419 0.05448254 0.97628083]]

 [[0.99047619 1.         0.98139535 0.09442552 1.        ]]]


In [ ]:
print("Data test_y")
print("=====================================")
print(test_y)

Data test_y
[1. 1.]


## Creating Hyperparameter Combination

In [ ]:
hyperparams = []
batch = [16, 32]
epoch = [50, 100]
neuron = [20,50]
for j in batch:
    for k in epoch:
        for l in neuron:
            hyperparams.append((j,k,l))
hyperparams

[(16, 50, 20),
 (16, 50, 50),
 (16, 100, 20),
 (16, 100, 50),
 (32, 50, 20),
 (32, 50, 50),
 (32, 100, 20),
 (32, 100, 50)]

In [ ]:
hyperparam1 = hyperparams[:4]
hyperparam2 = hyperparams[4:8]
hyperparam1

[(16, 50, 20), (16, 50, 50), (16, 100, 20), (16, 100, 50)]

In [ ]:
hyperparam2

[(32, 50, 20), (32, 50, 50), (32, 100, 20), (32, 100, 50)]

##Training Model

In [ ]:
lstms1 = []
models1 = []
for batch, epoch, neuron in hyperparam1:
    model, lstm = NeuralNetwork.train_lstm(train_X, train_y, test_X, test_y, neuron, epoch, batch)
    lstms1.append(lstm)
    models1.append(model)

Epoch 1/50
77/77 - 3s - loss: 0.1456 - val_loss: 0.0130 - 3s/epoch - 34ms/step
Epoch 2/50
77/77 - 0s - loss: 0.0519 - val_loss: 0.0443 - 176ms/epoch - 2ms/step
Epoch 3/50
77/77 - 0s - loss: 0.0405 - val_loss: 0.0264 - 181ms/epoch - 2ms/step
Epoch 4/50
77/77 - 0s - loss: 0.0315 - val_loss: 0.0193 - 179ms/epoch - 2ms/step
Epoch 5/50
77/77 - 0s - loss: 0.0231 - val_loss: 0.0313 - 177ms/epoch - 2ms/step
Epoch 6/50
77/77 - 0s - loss: 0.0181 - val_loss: 0.0282 - 171ms/epoch - 2ms/step
Epoch 7/50
77/77 - 0s - loss: 0.0155 - val_loss: 0.0233 - 188ms/epoch - 2ms/step
Epoch 8/50
77/77 - 0s - loss: 0.0145 - val_loss: 0.0212 - 173ms/epoch - 2ms/step
Epoch 9/50
77/77 - 0s - loss: 0.0142 - val_loss: 0.0196 - 192ms/epoch - 2ms/step
Epoch 10/50
77/77 - 0s - loss: 0.0142 - val_loss: 0.0212 - 186ms/epoch - 2ms/step
Epoch 11/50
77/77 - 0s - loss: 0.0144 - val_loss: 0.0198 - 174ms/epoch - 2ms/step
Epoch 12/50
77/77 - 0s - loss: 0.0142 - val_loss: 0.0197 - 180ms/epoch - 2ms/step
Epoch 13/50
77/77 - 0s - lo

## Testing Model

In [ ]:
i=0
wb1 = Workbook()
ws1 = wb1.active
for m in models1:
    # make a prediction
    test_x2 = test_X
    yhat = m.predict(test_x2)
    inv_yhat = Preprocessing.inverse_scaler(yhat, scaler)
    inv_y = Preprocessing.inverse_scaler(test_y.reshape(-1,1), scaler)
    print(hyperparam1[i])
    print("Epoch: "+ str(lstms1[i].params['epochs']))
    print("Neurons: "+str(m.layers[0].units))

    i = i+1
    ws1['A'+str(i)] = 'DWT-LSTM'
    ws1['B'+str(i)] = hyperparam1[i-1][0]
    ws1['C'+str(i)] = hyperparam1[i-1][1]
    ws1['D'+str(i)] = hyperparam1[i-1][2]
    print('RMSE')
    print(Evaluation.rmse(inv_y,inv_yhat))
    ws1['E'+str(i)] = Evaluation.rmse(inv_y,inv_yhat)

    print('MAE')
    print(Evaluation.mae(inv_y,inv_yhat))
    ws1['F'+str(i)] = Evaluation.mae(inv_y,inv_yhat)

    print('MAPE')
    print(Evaluation.mape(inv_y,inv_yhat))
    ws1['G'+str(i)] = Evaluation.mape(inv_y,inv_yhat)

    NeuralNetwork.save_model(m, 1, 'BBCA',hyperparam1[i-1])
    with open('DWT_LSTM_BBCA'+str(hyperparam1[i-1])+'.pkl', 'wb') as f:
        pickle.dump(lstms1[i-1].history, f)
wb1.save('DWT_LSTM_BBCA_result1.xlsx')

1/1 [==============================] - 0s 419ms/step
(16, 50, 20)
Epoch: 50
Neurons: 20
RMSE
111.93827874079439
MAE
108.0810546875
MAPE
1.1142376771907219


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 419ms/step
(16, 50, 50)
Epoch: 50
Neurons: 50
RMSE
61.398595050338294
MAE
49.31103515625
MAPE
0.508361187177835


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 1s 986ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


(16, 100, 20)
Epoch: 100
Neurons: 20
RMSE
119.53043782675084
MAE
114.81640625
MAPE
1.1836742912371134


1/1 [==============================] - 1s 529ms/step
(16, 100, 50)
Epoch: 100
Neurons: 50
RMSE
92.2848482783593
MAE
83.41357421875
MAPE
0.8599337548324741


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:

lstms2 = []
models2 = []
for batch, epoch, neuron in hyperparam2:
    model, lstm = NeuralNetwork.train_lstm(train_X, train_y, test_X, test_y, neuron, epoch, batch)
    lstms2.append(lstm)
    models2.append(model)

Epoch 1/50
39/39 - 4s - loss: 0.3207 - val_loss: 0.4832 - 4s/epoch - 92ms/step
Epoch 2/50
39/39 - 0s - loss: 0.0567 - val_loss: 0.0394 - 103ms/epoch - 3ms/step
Epoch 3/50
39/39 - 0s - loss: 0.1031 - val_loss: 0.0830 - 98ms/epoch - 3ms/step
Epoch 4/50
39/39 - 0s - loss: 0.0565 - val_loss: 0.0205 - 109ms/epoch - 3ms/step
Epoch 5/50
39/39 - 0s - loss: 0.0518 - val_loss: 0.0142 - 110ms/epoch - 3ms/step
Epoch 6/50
39/39 - 0s - loss: 0.0271 - val_loss: 0.0150 - 108ms/epoch - 3ms/step
Epoch 7/50
39/39 - 0s - loss: 0.0141 - val_loss: 0.0223 - 110ms/epoch - 3ms/step
Epoch 8/50
39/39 - 0s - loss: 0.0139 - val_loss: 0.0197 - 108ms/epoch - 3ms/step
Epoch 9/50
39/39 - 0s - loss: 0.0139 - val_loss: 0.0184 - 112ms/epoch - 3ms/step
Epoch 10/50
39/39 - 0s - loss: 0.0140 - val_loss: 0.0167 - 106ms/epoch - 3ms/step
Epoch 11/50
39/39 - 0s - loss: 0.0142 - val_loss: 0.0182 - 117ms/epoch - 3ms/step
Epoch 12/50
39/39 - 0s - loss: 0.0143 - val_loss: 0.0186 - 102ms/epoch - 3ms/step
Epoch 13/50
39/39 - 0s - los

In [ ]:
i=0
wb2 = Workbook()
ws2 = wb2.active
for m in models2:
    # make a prediction
    test_x2 = test_X
    yhat = m.predict(test_x2)
    inv_yhat = Preprocessing.inverse_scaler(yhat, scaler)
    inv_y = Preprocessing.inverse_scaler(test_y.reshape(-1,1), scaler)
    print(hyperparam2[i])
    print("Epoch: "+ str(lstms2[i].params['epochs']))
    print("Neurons: "+str(m.layers[0].units))

    i = i+1
    ws2['A'+str(i)] = 'DWT-LSTM'
    ws2['B'+str(i)] = hyperparam2[i-1][0]
    ws2['C'+str(i)] = hyperparam2[i-1][1]
    ws2['D'+str(i)] = hyperparam2[i-1][2]

    print('RMSE')
    print(Evaluation.rmse(inv_y,inv_yhat))
    ws2['E'+str(i)] = Evaluation.rmse(inv_y,inv_yhat)

    print('MAE')
    print(Evaluation.mae(inv_y,inv_yhat))
    ws2['F'+str(i)] = Evaluation.mae(inv_y,inv_yhat)

    print('MAPE')
    print(Evaluation.mape(inv_y,inv_yhat))
    ws2['G'+str(i)] = Evaluation.mape(inv_y,inv_yhat)

    NeuralNetwork.save_model(m, 1, 'BBCA',hyperparam2[i-1])
    with open('DWT_LSTM_BBCA'+str(hyperparam2[i-1])+'.pkl', 'wb') as f:
        pickle.dump(lstms2[i-1].history, f)
wb2.save('DWT_LSTM_BBCA_result2.xlsx')

1/1 [==============================] - 0s 421ms/step
(32, 50, 20)
Epoch: 50
Neurons: 20
RMSE
101.28152111738687
MAE
97.8408203125
MAPE
1.00866825064433


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 425ms/step
(32, 50, 50)
Epoch: 50
Neurons: 50
RMSE
61.28013353045634
MAE
52.8916015625
MAPE
0.5452742429123711


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 406ms/step
(32, 100, 20)
Epoch: 100
Neurons: 20
RMSE
97.71976740264458
MAE
93.38232421875
MAPE
0.9627043733891751


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 443ms/step
(32, 100, 50)
Epoch: 100
Neurons: 50
RMSE
96.00954979842545
MAE
90.328125
MAPE
0.9312177835051547


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
